In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.nn.utils import spectral_norm
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

def set_seeds(seed=42):
    torch.manual_seed(seed)
    torch.mps.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seeds()

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: mps


### Setup Dataset Class

In [16]:
class SensorDataset(Dataset):
    def __init__(self, X, y, session_ids, window_size=3):
        self.X = []
        self.y = []
        self.window_size = window_size

        # Group data by session
        session_groups = {}
        for idx, session_id in enumerate(session_ids):
            if session_id not in session_groups:
                session_groups[session_id] = []
            session_groups[session_id].append((X[idx], y[idx]))

        # Create sequences within each session
        for session_id, session_data in session_groups.items():
            # Create sliding windows of walking steps
            for i in range(len(session_data) - window_size + 1):
                # Option 1: Concatenate the walking steps into a single sequence
                # This creates a 2D tensor with shape [window_size * time_steps, num_sensors]
                steps = [data[0] for data in session_data[i:i+window_size]]
                window_X = np.vstack(steps)  # Stack vertically to create one long sequence
                
                # Use the label from the last step in the window
                window_y = session_data[i+window_size-1][1]
                
                self.X.append(window_X)
                self.y.append(window_y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)

### Load Data

In [17]:
X_original = np.load('sensor_data.npy')  # Shape: (num_steps, time_steps, num_sensors)
metadata = pd.read_csv('combined_metadata.csv')
y = metadata['has_ms'].values

### Split Data

In [18]:
# Split by session for sequence integrity while stratifying
sessions = metadata['session_id'].values
unique_sessions = np.unique(sessions)

# Create a mapping of session_id to MS status
session_to_ms_status = {}
for session_id in unique_sessions:
    # Get all rows for this session
    session_mask = metadata['session_id'] == session_id
    # If any row has MS, the whole session is labeled as MS
    has_ms = any(metadata.loc[session_mask, 'has_ms'] == 1)
    session_to_ms_status[session_id] = 1 if has_ms else 0

# Create lists of session IDs by MS status
ms_sessions = [s for s, status in session_to_ms_status.items() if status == 1]
non_ms_sessions = [s for s, status in session_to_ms_status.items() if status == 0]

# Perform stratified split on MS and non-MS sessions separately
train_ms, temp_ms = train_test_split(ms_sessions, test_size=0.3, random_state=42, shuffle=True)
train_non_ms, temp_non_ms = train_test_split(non_ms_sessions, test_size=0.3, random_state=42, shuffle=True)

# Further split temp sets into validation and test
val_ms, test_ms = train_test_split(temp_ms, test_size=0.5, random_state=42, shuffle=True)
val_non_ms, test_non_ms = train_test_split(temp_non_ms, test_size=0.5, random_state=42, shuffle=True)

# Combine MS and non-MS sessions for each split
train_sessions = train_ms + train_non_ms
val_sessions = val_ms + val_non_ms
test_sessions = test_ms + test_non_ms

train_indices = metadata['session_id'].isin(train_sessions)
val_indices = metadata['session_id'].isin(val_sessions)
test_indices = metadata['session_id'].isin(test_sessions)

In [19]:
# Downsample X to have 10 timesteps instead of 100
X = X_original.copy()#reshape(X_original.shape[0], 25, -1, X_original.shape[2]).mean(axis=2)

X_train, X_val, X_test = X[train_indices], X[val_indices], X[test_indices]
y_train, y_val, y_test = y[train_indices], y[val_indices], y[test_indices]

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X.shape[2])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X.shape[2])).reshape(X_val.shape)
X_test = scaler.transform(X_test.reshape(-1, X.shape[2])).reshape(X_test.shape)

# Session IDs for reference
train_sessions_ids = sessions[train_indices]
val_sessions_ids = sessions[val_indices]
test_sessions_ids = sessions[test_indices]

window_size = 4

# Create datasets
train_dataset = SensorDataset(X_train, y_train, train_sessions_ids, window_size=window_size)
val_dataset = SensorDataset(X_val, y_val, val_sessions_ids, window_size=window_size)
test_dataset = SensorDataset(X_test, y_test, test_sessions_ids, window_size=window_size)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

### Traditional

In [20]:
import numpy as np
from scipy import stats
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from scipy import signal

def extract_features(dataset):
    """Extract statistical features from time series data."""
    features = []
    labels = []
    
    for i in range(len(dataset)):
        X, y = dataset[i]
        X = X.numpy()  # Convert from tensor to numpy
        
        # For each sequence, compute statistical features
        sequence_features = []
        
        # Statistical features for each sensor
        for j in range(X.shape[1]):  # For each sensor
            sensor_data = X[:, j]
            
            # Basic statistics
            freqs, psd = signal.welch(sensor_data, fs=100)

            sequence_features.extend([
                np.mean(sensor_data),        # Mean
                np.std(sensor_data),         # Standard deviation
                np.min(sensor_data),         # Minimum
                np.max(sensor_data),         # Maximum
                np.median(sensor_data),      # Median
                stats.skew(sensor_data),     # Skewness
                stats.kurtosis(sensor_data), # Kurtosis
                np.percentile(sensor_data, 25),  # 25th percentile
                np.percentile(sensor_data, 75),  # 75th percentile
                np.ptp(sensor_data),         # Range (peak-to-peak)
                np.sum(psd),            # Total power
                np.mean(psd),           # Average power
                np.max(psd),            # Peak frequency power
                freqs[np.argmax(psd)]   # Dominant frequency
            ])

        # Add features for trends/dynamics
        for j in range(X.shape[1]):  # For each sensor
            sensor_data = X[:, j]
            if len(sensor_data) > 5:  # Need sufficient data points
                # Linear trend
                detrended = signal.detrend(sensor_data)
                trend = sensor_data - detrended
                sequence_features.append(np.mean(trend))
                
                # First and second derivatives
                first_diff = np.diff(sensor_data)
                sequence_features.extend([
                    np.mean(np.abs(first_diff)),   # Mean absolute change
                    np.std(first_diff),            # Variability of change
                ])
                
                if len(first_diff) > 1:
                    second_diff = np.diff(first_diff)
                    sequence_features.append(np.mean(np.abs(second_diff)))  # Acceleration
        
        features.append(sequence_features)
        labels.append(y.item())  # Convert from tensor to scalar
    
    return np.array(features), np.array(labels)

# Extract features
X_train_features, y_train = extract_features(train_dataset)
X_val_features, y_val = extract_features(val_dataset)
X_test_features, y_test = extract_features(test_dataset)

print(f"Extracted {X_train_features.shape[1]} features per sequence")

Extracted 126 features per sequence


In [21]:
print(X_train.shape)
print(X_train_features.shape)

(10567, 100, 7)
(10128, 126)


In [22]:
def add_mean_duration_feature(X_features, metadata, session_ids):
    # Filter session_ids to match the number of rows in X_features
    session_ids = session_ids[:len(X_features)]

    # Create an array to hold mean durations for each step
    mean_durations = np.zeros(len(X_features))
    
    # Map session-level duration to each sample
    for session_id in np.unique(session_ids):
        session_mask = session_ids == session_id
        session_duration = metadata.loc[metadata['session_id'] == session_id, 'duration'].mean()

        # Fill mean duration for each sample in this session
        mean_durations[session_mask] = session_duration

    # Reshape for concatenation
    mean_durations = mean_durations.reshape(-1, 1)
    
    # Append mean duration as a new feature
    return np.hstack((X_features, mean_durations))

# Corrected Data Integration
X_train_features = add_mean_duration_feature(X_train_features, metadata, train_sessions_ids)
X_val_features = add_mean_duration_feature(X_val_features, metadata, val_sessions_ids)
X_test_features = add_mean_duration_feature(X_test_features, metadata, test_sessions_ids)


In [24]:
X_combined = np.vstack((X_train_features, X_val_features))
y_combined = np.concatenate((y_train, y_val))

# Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 1.0}
optimized_gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.2,
    max_depth=3,
    min_samples_split=2,
    random_state=42
)

optimized_gb.fit(X_combined, y_combined)
y_test_pred = optimized_gb.predict(X_test_features)

# Calculate metrics
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Accuracy: 0.9497
Precision: 0.8906
Recall: 0.9683
F1 Score: 0.9278

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96      1509
           1       0.89      0.97      0.93       757

    accuracy                           0.95      2266
   macro avg       0.94      0.95      0.94      2266
weighted avg       0.95      0.95      0.95      2266



In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate

# Combine training and validation data
X_combined = np.vstack((X_train_features, X_val_features))
y_combined = np.concatenate((y_train, y_val))

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_validate(
    optimized_gb, 
    X_combined, 
    y_combined, 
    cv=cv,
    scoring=['accuracy', 'precision', 'recall', 'f1']
)

# Print results
print("Cross-Validation Results (5 folds):")
print(f"Accuracy: {cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}")
print(f"Precision: {cv_results['test_precision'].mean():.4f} ± {cv_results['test_precision'].std():.4f}")
print(f"Recall: {cv_results['test_recall'].mean():.4f} ± {cv_results['test_recall'].std():.4f}")
print(f"F1 Score: {cv_results['test_f1'].mean():.4f} ± {cv_results['test_f1'].std():.4f}")

Cross-Validation Results (5 folds):
Accuracy: 0.9980 ± 0.0007
Precision: 0.9974 ± 0.0023
Recall: 0.9971 ± 0.0023
F1 Score: 0.9972 ± 0.0009


#### GB Regular CV:
Accuracy: 0.8883 ± 0.0505
Precision: 0.8854 ± 0.0345
Recall: 0.7929 ± 0.1382
F1 Score: 0.8308 ± 0.0894

In [ ]:
import pickle

# Save the model to a file
with open('optimized_gb_model.pkl', 'wb') as file:
    pickle.dump(optimized_gb, file)

print("Model saved to 'optimized_gb_model2.pkl'")

Model saved to 'optimized_gb_model.pkl'


In [42]:
import numpy as np
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Combine training and validation sets
X_combined = np.vstack((X_train_features, X_val_features))
y_combined = np.concatenate((y_train, y_val))

# Define the base estimator with your tuned parameters
base_gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.2,
    max_depth=3,
    min_samples_split=2,
    random_state=42
)

# Create an ensemble of multiple instances of your GradientBoostingClassifier
ensemble_gb = BaggingClassifier(
    estimator=base_gb,
    n_estimators=5,          # number of copies
    max_samples=0.8,         # each model trains on 80% of the combined data (bootstrapped)
    bootstrap=True,
    random_state=42
)

# Train the ensemble on the combined training set
ensemble_gb.fit(X_combined, y_combined)

# Evaluate on the test set
y_test_pred = ensemble_gb.predict(X_test_features)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print(f"Ensemble Accuracy: {accuracy:.4f}")
print(f"Ensemble Precision: {precision:.4f}")
print(f"Ensemble Recall: {recall:.4f}")
print(f"Ensemble F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Ensemble Accuracy: 0.9612
Ensemble Precision: 0.9166
Ensemble Recall: 0.9723
Ensemble F1 Score: 0.9436

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      1509
           1       0.92      0.97      0.94       757

    accuracy                           0.96      2266
   macro avg       0.95      0.96      0.96      2266
weighted avg       0.96      0.96      0.96      2266

